In [1]:
import cv2 as cv

In [193]:
metodo = 'MOG2'
#metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = False)

#### Corremos la sustracción de fondo

In [172]:

import numpy as np
class BackgroundSubtractorMedian:
    
    def __init__(self,filename, initKframes = 10, update_rate = 100):
        self.count = 0
        self.update_rate = update_rate
        capture = cv.VideoCapture(filename)

        self.frames_for_update = []

        for i in range(initKframes):
            ret, frame = capture.read()
            if frame is None:
                break
            else:
                frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
                self.frames_for_update.append(frame)
    
        self.median = np.median(self.frames_for_update, axis=0)

    def apply(self, frame):
        self.frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
        self.count += 1
        self.frames_for_update.append(self.frame)
        self.mask = cv.threshold(np.clip(self.median-self.frame,0,255),64,1,cv.THRESH_BINARY)[1]
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
        self.mask = cv.morphologyEx(self.mask, cv.MORPH_CLOSE, kernel)
        self.mask = cv.erode(self.mask,kernel)

        if self.count // self.update_rate == 1:
            self.count = 0
            self.update()
            self.frames_for_update = []
        
        return self.mask

    
    def update(self):
        new = np.median(self.frames_for_update, axis=0)
        self.median =  new






In [203]:
# Abrimos el archivo
#-------------------
filename = 'vtest.avi'
BSM = BackgroundSubtractorMedian(filename, initKframes = 50, update_rate = 200)

capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# Corremos la sustraccion
#------------------------

while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    mask = BSM.apply(frame)
    
    cv.imshow('masked_Frame', np.array(frame * mask[:,:,np.newaxis], np.uint8))       
    cv.imshow('original', frame)
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break
cv.destroyAllWindows()


In [201]:
# Abrimos el archivo
#-------------------
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    fgMask = backSub.apply(frame)
    mask = BSM.apply(frame)
    

    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    # cv.imshow('Frame', frame)

    cv.imshow('MGO', fgMask)
    cv.imshow('Median', mask)
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

## conclusiones


Se implemento una clase que permite eliminar el background de un video para detectar los objetos en movimiento. El método se basa en eliminación por mediana.

Los resultados obtenidos comparados con el método basado en MOG muestran un buen desempeño en la detección de los objetos, perdiéndose en algunos casos particulares la silueta de las personas. Sin embargo, el método implementado no presento grandes indicios de falsos positivos y la actualización de la mediana no fue necesaria con tanta frecuencia. El método basado en MOG como fue utilizado en clase, muestra una marcada detección de la silueta de las personas, pero agrega mucho ruido a la máscara. Este ruido podría ser mitigado aplicando transformaciones morfológicas como erode con kernel pequeño. Las sombras de las personas es un punto a trabajar, si representa un problema en la aplicación.

En ambos casos, un factor a tener en cuenta es la capacidad de procesamiento que se cuenta y el tiempo requerido para aplicar las transformaciones morfológicas necesarias.